In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import folium
import math

In [2]:
# list of dates for the downloaded files
lstDates = ['202001','202002','202003','202004','202005','202006','202007','202008','202009','202010','202011','202012']

In [3]:
lstdf = []
# a loop to read the monthly csv files 
for date in lstDates:
    try:
        # read each month's csv
        pathname = r"C:\Users\merjl\OneDrive - CUNY\Desktop\CitiBikeFilesUrbanGIS\2020\{}-citibike-tripdata.csv\{}-citibike-tripdata.csv" .format(date,date)
        # add (append) this to a list
        lstdf.append(pd.read_csv(pathname))
    except:
        print('got an error for ',date)

In [4]:
# put together all monthly dataframe and create one big dataframe
dfall = pd.concat(lstdf)

In [5]:
dfall.sample(4)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
359099,2501,2020-06-08 14:15:16.6430,2020-06-08 14:56:57.9170,3043,Lewis Ave & Decatur St,40.681460,-73.934903,363,West Thames St,40.708347,-74.017134,40826,Customer,1995,2
1352728,307,2020-11-22 15:07:57.5610,2020-11-22 15:13:05.1110,3164,Columbus Ave & W 72 St,40.777057,-73.978985,3697,W 64 St & Thelonious Monk Circle,40.775160,-73.989187,45559,Subscriber,1992,1
463291,2666,2020-08-07 15:52:30.0770,2020-08-07 16:36:56.5600,3165,Central Park West & W 72 St,40.775794,-73.976206,3374,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955613,45599,Subscriber,1977,1
1781225,759,2020-09-22 17:54:58.9540,2020-09-22 18:07:38.2250,499,Broadway & W 60 St,40.769155,-73.981918,3287,W 87 St & West End Ave,40.789622,-73.977570,33023,Subscriber,1984,1


In [6]:
dfall.info(4)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19506857 entries, 0 to 1088928
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(4), int64(6), object(5)
memory usage: 2.3+ GB


In [7]:
#Remove extra columns
dfselect = dfall.filter(['start station id','start station latitude','start station longitude', 'end station id', 'end station latitude', 'end station longitude'], axis=1)

In [8]:
dfselect.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19506857 entries, 0 to 1088928
Data columns (total 6 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   start station id         int64  
 1   start station latitude   float64
 2   start station longitude  float64
 3   end station id           int64  
 4   end station latitude     float64
 5   end station longitude    float64
dtypes: float64(4), int64(2)
memory usage: 1.0 GB


In [9]:
#Create a dataframe of only the end station id and start station id
dfIDonly = dfselect.filter(['start station id','end station id'], axis =1)

In [10]:
dfIDonly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19506857 entries, 0 to 1088928
Data columns (total 2 columns):
 #   Column            Dtype
---  ------            -----
 0   start station id  int64
 1   end station id    int64
dtypes: int64(2)
memory usage: 446.5 MB


In [11]:
#Count how many times each combination of station id occurs into a new dataframe
dfIDonly["count"] = 1
dfIDgroup = dfIDonly.groupby(["start station id", "end station id"])["count"].count().reset_index()

In [12]:
dfIDgroup.sample(20)

,start station id,end station id,count
99682,391,278,109
224556,3093,3641,6
448432,3709,3424,38
70611,334,3690,41
461273,3737,3820,4
602540,4232,3314,8
182639,533,372,1
249546,3141,3335,3
429960,3630,4120,1
4419,127,376,51


In [13]:
dfIDgroup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604217 entries, 0 to 604216
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   start station id  604217 non-null  int64
 1   end station id    604217 non-null  int64
 2   count             604217 non-null  int64
dtypes: int64(3)
memory usage: 13.8 MB


In [14]:
dfIDgroup.sum()

start station id    1542403547
end station id      1547388159
count                 19506857
dtype: int64

In [15]:
#Create a separate  dataframe of the start id and coordinates from the full dataframe
dfStarts = (dfall[['start station id','start station latitude','start station longitude']]).groupby(['start station id']).agg({'start station latitude':['first'],'start station longitude':['first']})

In [16]:
dfStarts = dfStarts.droplevel(1,axis=1).reset_index()

In [17]:
dfStarts = pd.DataFrame(dfStarts)

In [18]:
dfStarts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   start station id         1213 non-null   int64  
 1   start station latitude   1213 non-null   float64
 2   start station longitude  1213 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 28.6 KB


In [19]:
#index match thw two dataframes with the start station id to create a dataframe that contains the corosponding start coordinates 
dfstertsmerged= dfIDgroup.merge(dfStarts,how='left',on='start station id')

In [20]:
dfstertsmerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 604217 entries, 0 to 604216
Data columns (total 5 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   start station id         604217 non-null  int64  
 1   end station id           604217 non-null  int64  
 2   count                    604217 non-null  int64  
 3   start station latitude   604217 non-null  float64
 4   start station longitude  604217 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 27.7 MB


In [21]:
dfstertsmerged.sample(10)

,start station id,end station id,count,start station latitude,start station longitude
61284,317,3572,3,40.724537,-73.981854
58091,312,2017,27,40.722055,-73.989111
440749,3686,280,149,40.739448,-74.005070
376349,3505,3387,16,40.805726,-73.936322
2204,83,3315,31,40.683826,-73.976323
58166,312,3128,1,40.722055,-73.989111
602919,4236,146,2,40.768333,-73.992573
102068,394,3087,9,40.725213,-73.977688
94429,380,4070,2,40.734011,-74.002939
532069,3913,3578,6,40.699569,-73.979827


In [22]:
#Create a separate  dataframe of the end id and coordinates from the full dataframe
dfEnds = (dfall[['end station id','end station latitude','end station longitude']]).groupby(['end station id']).agg({'end station latitude':['first'],'end station longitude':['first']})

In [23]:
dfEnds = dfEnds.droplevel(1,axis=1).reset_index()

In [24]:
dfEnds = pd.DataFrame(dfEnds)

In [25]:
#index match thw two dataframes with the start end station id to create a dataframe that contains the corosponding end coordinates 
dfTotalsmergedRoutes2019= dfstertsmerged.merge(dfEnds,how='left',on='end station id')

In [83]:
# the end results is a dataframe that contains the count of
dfTotalsmergedRoutes2019.sample()

,start station id,end station id,count,start station latitude,start station longitude,end station latitude,end station longitude
206174,3137.0,377.0,15,40.772828,-73.966853,40.722438,-74.005664


In [25]:
dfTotalsmergedRoutes2019.to_csv(r'C:\Users\merjl\OneDrive - CUNY\Desktop\CitiBikeFilesUrbanGIS\2020Routes.csv', index = False)